In [ ]:
using Flux, Flux.Data.FashionMNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra
train_labels = FashionMNIST.labels();
train_imgs = FashionMNIST.images();

In [2]:
epochs = 10;
batch_size = 128;

In [ ]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

In [ ]:
mb_idxs = partition(1:length(train_imgs), batch_size)
train_set = [make_minibatch(train_imgs, train_labels, i) for i in mb_idxs];

In [ ]:
# Prepare test set as one giant minibatch:
test_imgs = FashionMNIST.images(:test)
test_labels = FashionMNIST.labels(:test)
test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs));

In [30]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>64, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 64=>128, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 128=>128, pad=(1,1), relu),
    MaxPool((2,2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(128, 10),
    softmax,
);

In [34]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>64, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 64=>128, pad=(1,1), relu),
    MaxPool((2,2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(128, 10),
    softmax,
);

In [ ]:
#nowa próba
model = Chain(
    Conv((5, 5), 1=>32, pad=(2,2), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>64, pad=(2,2), relu),
    MaxPool((2,2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(4096, 10),
    softmax,
);

In [13]:
con = Chain(model.layers[1:7]...)

Chain(Conv((5, 5), 1=>32, relu), BatchNorm(32), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), BatchNorm(64), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), #9)

In [ ]:
con(train_set[1][1])

In [ ]:
model(train_set[1][1])

In [ ]:
function loss(x, y)
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x))
    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end

In [8]:
loss(train_set[1][1],train_set[1][2])

2.2995205f0

In [ ]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

In [52]:
accuracy(train_set[1][1],train_set[1][2])

0.0546875

In [1]:
opt = ADAM(0.001)

UndefVarError: UndefVarError: ADAM not defined

In [54]:
Flux.train!(loss, params(model), train_set, opt)

InterruptException: InterruptException:

In [42]:
BSON.@load "fashionMNIST_conv3.bson" model

In [ ]:
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch = 1:epochs
    global best_acc, last_improvement
    Flux.train!(loss, params(model), train_set, opt)
    acc = accuracy(test_set[1],test_set[2])
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))
    if acc >= 0.91
        @info(" -> Early-exiting: We reached our target accuracy of 91%")
        break
    end
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to fashionMNIST_conv.bson")
        BSON.@save "fashionMNIST_conv3.bson" model epoch acc
        best_acc = acc
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end